# Meteor Strikes

- **_Meteoroid_**: An asteroid or comet fragment that orbits the Sun (< 1m).

- **_Meteor_**, or “shooting star”: The visible paths of meteoroids that have entered the Earth’s atmosphere at a high velocity.

- **_Fireball_**: An unusually bright meteor that reaches a visual magnitude of -3 or brighter when seen at the observer’s zenith (> 1m).

- **_Bolide_**: Fireballs that explode in the atmosphere, although the terms fireballs and bolides are often used interchangeably.

- **_Meteorite_**: A fragment of a meteoroid or asteroid that survives passage through the Earth’s atmosphere and hits the ground.

<img src="http://www.amsmeteors.org/wp-content/uploads/2010/11/AMS-TERMINOLOGY-2015-EN-ENGLISH-compressed3.jpg" alt="drawing" width="300"/>

Meteors burn up very fast, as quoted from [article](https://pawsey.org.au/case_studies/catching-shooting-stars/):
> In November 2015, an 80-kilogram meteoroid that had been moving at 50 thousand kilometres per hour entered the atmosphere. It created a fireball that streaked across the sky; becoming what some of us refer to as a shooting star. The meteorite then fell into a remote area of South Australia as a 1.6 kilogram of mottled rock.

Natural disasters from [meteorite impacts in history](https://science.nasa.gov/solar-system/meteors-meteorites/facts/#h-meteorite-impacts-in-history):
* Tunguska event of 1908: About 37 meters across and weighed 100 million kilograms
* Ann Hodges of Sylacauga, Alabama, was severely bruised by a 3.6-kilogram stony meteorite that crashed through her roof in November 1954.
* Chelyabinsk, Russia in 2013: 440,000 tons of TNT-equivalent fireball exploded 23km above ground.
    * [Video link](https://youtu.be/mebWfDlhcRs)

How frequently do fireballs occur?
* Several thousand meteors of fireball magnitude occur in the Earth’s atmosphere each day.
* The vast majority of these, however, occur over the oceans and uninhabited regions, and a good many are masked by daylight.
* Those that occur at night also stand little chance of being detected due to the relatively low numbers of persons out to notice them.
* Brighter fireballs are more rare events.

# Meteorite Landings
**_Dataset_**: https://data.nasa.gov/dataset/meteorite-landings

How are meteorites discovered?
* **Fell** - Meteorite was seen while falling, which led to to its discovery (Date is accurate).
* **Found** - Meteorite was found lying on the ground (Date of fall is _unknown_).

In [1]:
import polars as pl
import plotly.express as plt
import plotly.io

# plotly.io.renderers.default = "iframe"

In [2]:
meteorites = (
    pl.read_csv("../data/nasa/Meteorite_Landings.csv")
    .rename({"mass (g)": "mass"})
    .select("id", "year", "fall", pl.col("mass") / 1000)
    .filter(pl.col("mass").is_not_null())
    .filter((pl.col("year") >= 1800) & (pl.col("year") < 2026))
    .sort("year")
)

meteorites

id,year,fall,mass
i64,i64,str,f64
15447,1801,"""Fell""",0.22
2320,1803,"""Fell""",3.2
12434,1803,"""Fell""",37.0
15443,1803,"""Fell""",1.6
5093,1804,"""Fell""",0.056
…,…,…,…
57422,2013,"""Found""",0.246
57423,2013,"""Found""",0.459
57425,2013,"""Found""",0.611


<sub>(*mass in kg)</sub>

Largest 10 meteorites discovered (up to 2013):

In [3]:
meteorites.sort("mass", descending=True).head(10)

id,year,fall,mass
i64,i64,str,f64
11890,1920,"""Found""",60000.0
5262,1818,"""Found""",58200.0
5257,1891,"""Found""",30000.0
2335,1898,"""Found""",28000.0
10912,1836,"""Found""",26000.0
5363,1852,"""Found""",24300.0
16852,1911,"""Found""",24000.0
23593,1947,"""Fell""",23000.0
4919,1863,"""Found""",22000.0


In [4]:
meteorites.group_by("fall").agg(
    pl.len().alias("count"),
    pl.mean("mass").alias("mean"),
    pl.max("mass").alias("max"),
)

fall,count,mean,max
str,u32,f64,f64
"""Found""",44222,10.891534,60000.0
"""Fell""",1040,48.15393,23000.0


Summarised below is the **annual block maxima** i.t.o. mass:

In [5]:
meteorite_max = meteorites.group_by(("fall", "year")).agg(pl.max("mass"), pl.len().alias("count")).sort("year")

plt.line(meteorite_max, "year", "mass", color="fall", log_y=True, markers=True, title="Yearly Maxima of Mass") \
    .update_traces(marker_symbol=1, marker_size=4, line_width=1) \
    .update_layout(legend_title="Discovery", yaxis_title_text="Mass (kg)") \
    .show()

In [6]:
plt.line(meteorite_max, "year", "count", color="fall", markers=True, title="Yearly Counts") \
    .update_traces(marker_symbol=1, marker_size=4, line_width=1) \
    .update_layout(legend_title = "Discovery") \
    .show()

# NASA Fireballs
**_Dataset_**: https://cneos.jpl.nasa.gov/fireballs/

Goal is to answer the following **research questions**:
- Probability of another 440kt or larger meteor hitting the Earth?
- Probability of another 440kt or larger meteor hitting the Earth in one year?
- What proportion of all meteors exceed 440kt?
- How many meteorites before we'd expect at least one 440kt meteor?
- Larger meteorites?
- How reliably can we estimate deep quantiles?
- Tail behaviour as a way of understanding diminishing sizes of fireballs (EVI Index?)

Who might be interested in this research?
- Disaster Respondents
- Deep Space Observatories
- Geologists
- Astronomers

In [7]:
fireballs = (
    pl.read_csv(
        "../data/cneos/cneos_fireball_data.csv",
        new_columns=("time", "lat", "long", "alt", "vx", "vy", "vz", "E0", "E")
    )
    .with_columns(
        pl.col("time").str.to_datetime(),
        pl.col("alt", "vx", "vy", "vz").cast(pl.Float64, strict=False),
        pl.col("E").log().alias("logE")
    )
    .sort("time")
)

fireballs.select("time", "E", "logE").sort("E")

time,E,logE
datetime[μs],f64,f64
2020-02-10 23:48:17,0.048,-3.036554
2021-12-23 21:27:58,0.064,-2.748872
2011-05-16 13:39:06,0.065,-2.733368
2019-05-12 22:41:48,0.07,-2.65926
2022-03-03 00:03:02,0.071,-2.645075
…,…,…
2009-11-21 20:53:29,20.0,2.995732
1994-02-01 22:38:09,30.0,3.401197
2009-10-08 02:57:14,33.0,3.496508


In [8]:
(
    plt.scatter(
        fireballs,
        "time",
        "E",
        log_y=True,
        title="Estimated Impact Energy of Observed Fireballs",
        subtitle=f"N = {len(fireballs)}",
    )
    .update_traces(marker_symbol="x", marker_size=4, line_width=1)
    .update_layout(yaxis_title="Impact Energy (kt)", xaxis_title="Date")
    # .write_image("../resources/images/scatter.png", width=960, height=480, scale=4)
    .show()
)

In [9]:
plt.histogram(fireballs, "logE", title="Histogram of Impact Energies") \
    .update_layout(xaxis_title="Impact Energy (log-kt)", yaxis_title="Frequency") \
    # .write_image("../resources/images/hist.png", width=960, height=480, scale=4)

In [13]:
fireballs_quarterly = (
    fireballs.group_by_dynamic("time", every="1q")
    .agg(
        pl.max("E"),
        pl.max("logE"), 
        pl.len().alias("count")
    )
    .rename({"time": "quarter"})
)

(    
    plt.line(
        fireballs_quarterly,
        "quarter",
        "logE",
        markers=True,
        title="Quarterly Maxima of Impact Energies",
        subtitle=f"N = {len(fireballs_quarterly)}",
    )
    .update_traces(marker_symbol=1, marker_size=4, line_width=1)
    .update_layout(yaxis_title="Max Impact Energy (log-kt)", xaxis_title="Quarter")
    # .write_image("../resources/images/quarterly.png", width=960, height=480, scale=4)
    .show()
)

In [11]:
plt.bar(fireballs_quarterly, "quarter", "count", title="Number of Fireballs per Quarter") \
    .update_layout(yaxis_title="Frequency", xaxis_title="Quarter") \
    # .write_image("../resources/images/quarterly-count.png", width=960, height=480, scale=4)

* Perhaps drop < 1993-10-01?

In [ ]:
# (
#     # fireballs_quarterly.filter(pl.col.quarter >= pl.datetime(1993, 10, 1))
#     fireballs_quarterly
#     .write_csv("../data/cneos/cneos_fireball_quarterly.csv")
# )

In [ ]:
(
    fireballs_quarterly.write_csv("../data/cneos/cneos_fireball_quarterly.csv")
)

In [30]:
(
    fireballs.filter(pl.col.time >= pl.datetime(1993, 10, 1))
        .group_by_dynamic("time", every="18mo")
        .agg(
            pl.max("E"),
            pl.max("logE"), 
            pl.len().alias("count")
        )
        .write_csv("../data/cneos/cneos_fireball_18m.csv")
)

# Other Datasets
* [Meteoritical Bulletin Database](https://www.lpi.usra.edu/meteor/metbull.cfm)
* [NASA Bolide Detections](https://neo-bolide.ndc.nasa.gov/#/)
* [Global Meteor Network](https://globalmeteornetwork.org/about/)

# Further Reading
* [Catching Shooting Stars](https://pawsey.org.au/case_studies/catching-shooting-stars/)
* [Meteorite finds, falls, and a bit of statistics](https://earthandsolarsystem.wordpress.com/2020/09/01/meteorite-finds-falls-and-a-bit-of-statistics/)
* [Fireball FAQS](https://www.amsmeteors.org/fireballs/faqf/)
* [Pre-Impact Detections](https://cneos.jpl.nasa.gov/pi/)